In [1]:
import time
import numpy as np
from constants import *
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from build_model import model_tools

X = np.load('Norm_MinMax_%d_X.npy' % window_size)
y = np.load('Norm_MinMax_%d_y.npy' % window_size)

# le = LabelEncoder()
# le.fit(y)
# le_y = np.array(le.transform(y))
# le_y = le_y.reshape((-1, 1))

# ohe = OneHotEncoder()
# ohe.fit(le_y)

# y = ohe.transform(le_y).toarray()

In [2]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
                X.reshape(X.shape[0], X.shape[1]*X.shape[2]), y, test_size=0.3, random_state=2021)

In [7]:
# Import các thư viện
import csv
import pandas as pd
import numpy as np
from sklearn import svm
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, log_loss

s_acc = s_loss = 0

def scorer(model, X_test, y_test):
    global s_acc, s_loss
    y_pred = model.predict(X_test)
    s_acc += accuracy_score(y_test, y_pred)

    y_pred = model.predict_proba(X_test)
    s_loss += log_loss(y_test, y_pred)
    return s_acc

def score_dataset(dataset_name, model, X, y, cv=None):
    global s_acc, s_loss
    if cv:
        s_acc = s_loss = 0
        scores = cross_val_score(model, X, y, cv=cv, scoring=scorer)
        mean_acc = s_acc/cv
        mean_loss = s_loss/cv
        print("%12s %2d-folds = %.3f with loss = %.3f" % (dataset_name, cv, mean_acc, mean_loss))
        return mean_acc, mean_loss
    else:
        for nFold in range(2, 10+1):
            scores = cross_val_score(model, X, y, cv=nFold)
            print("%s %2d-folds = %.3f" % (dataset_name, nFold, np.mean(scores)))

In [8]:
import pandas as pd
from sklearn import tree
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier

models = {
    'kNN': KNeighborsClassifier(n_neighbors=2),
    'GaussianNB': GaussianNB(),
    'DecisionTree': tree.DecisionTreeClassifier(criterion="gini"),
    'Bagging': BaggingClassifier(base_estimator=tree.DecisionTreeClassifier(), n_estimators=100),
    'AdamBoost': AdaBoostClassifier(base_estimator=tree.DecisionTreeClassifier(max_depth=2), n_estimators=100),
    'RandomForest': RandomForestClassifier(n_estimators=50),
    'SVM_Linear': svm.SVC(kernel='linear', C=1000, probability=True),
    'SVM_Poly': svm.SVC(kernel='poly', C=100000, probability=True),
    'SVM_RBF': svm.SVC(kernel='rbf', C=100000, gamma=0.01, probability=True),
    'SVM_Sigmoid': svm.SVC(kernel='sigmoid', C=100000, gamma=0.0001, probability=True)
}

In [4]:
from sklearn import svm
from sklearn.metrics import accuracy_score

clf = svm.SVC(kernel='rbf', C=100000, gamma=0.01, probability=True)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

# Compute Accuracy
acc = accuracy_score(y_test, y_pred)
print('\nAccuracy:', acc*100)


Accuracy: 95.03105590062113


In [9]:
for model in models.items():
    mean_acc, mean_loss = score_dataset(model[0], model[1], X.reshape(X.shape[0], X.shape[1]*X.shape[2]), y, cv=10)

         kNN 10-folds = 0.927 with loss = 1.792
  GaussianNB 10-folds = 0.885 with loss = 2.743
DecisionTree 10-folds = 0.791 with loss = 7.230
     Bagging 10-folds = 0.871 with loss = 0.325
   AdamBoost 10-folds = 0.916 with loss = 0.378
RandomForest 10-folds = 0.895 with loss = 0.292
  SVM_Linear 10-folds = 0.904 with loss = 0.663
    SVM_Poly 10-folds = 0.901 with loss = 0.682
     SVM_RBF 10-folds = 0.906 with loss = 0.752
 SVM_Sigmoid 10-folds = 0.906 with loss = 0.595


In [5]:
from joblib import dump
dump(clf, 'svm.model')

['svm.model']

In [ ]:
model = model_tools(X, y, test_size=0.4, model_name='svm', n_layers=2, dropout=0.5)

In [ ]:
if not os.path.isdir(result_dir):  # For training
    os.mkdir(result_dir)
model.fit_and_save_model(es=False, mc=False, rlr=True, log=True)

In [ ]:
model.plt_fig()